In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components


root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,1))
#specify which erp we are analyzing
erps = 'lrp'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [ ]:
#setting up path/ directory
#access specific file for specific participant
def load_data_array(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    ndat = epochs.get_data()

    return ndat, pp_directory

In [2]:
#setting up path/ directory
#access specific file for specific participant
def load_data_epochs(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    #ndat = epochs.get_data()

    return epochs, pp_directory

In [3]:
#set up parameters for time-frequency morlet convolution
def set_tfr_params(l_freq = 6, h_freq = 35, num_freq = 50, cycle = 6, samp_rate = 200):
    freqs = np.logspace(*np.log10([l_freq, h_freq]), num = num_freq)
    n_cycles = cycle
    sfreq = samp_rate
    
    return freqs, n_cycles, sfreq

In [4]:
#save tfr'd data
def output_tfr_array(pp_num, data, ppdir, output):
    # Save the cleaned data :)
    out_fname = os.path.join(ppdir, 'p%03d_%s_tfr_array' % (pp_num, output))
    data.save(out_fname, overwrite=True)
    

In [ ]:
#create array (.npy files) separately for power and phase of tfr'd data, such that each participant will have two files
# repeat for other time-locked event (coded as erps) interested in: LRP, FRN
for pp in participants:
    data, ppdir = load_data_array(pp_num = pp, root_dir = root, erp_path = erps)
    freqs, n_cycles, sfreq = set_tfr_params()
    outputtypes = ['power']#, 'phase']

    for otype in range(0, len(outputtypes)):
        tf_dat = mne.time_frequency.tfr_array_morlet(
            data,
            sfreq=sfreq,
            freqs=freqs,
            n_cycles=n_cycles,
            use_fft=False,
            decim=1,
            n_jobs=None,
            output = outputtypes[otype]
        )
        
        output_tfr_array(pp, tf_dat, ppdir, output = outputtypes[otype])
        

In [5]:
#create array (.npy files) separately for power and phase of tfr'd data, such that each participant will have two files
# repeat for other time-locked event (coded as erps) interested in: LRP, FRN
for pp in participants:
    data, ppdir = load_data_epochs(pp_num = pp, root_dir = root, erp_path = erps)
    freqs, n_cycles, sfreq = set_tfr_params()
    outputtypes = ['power']#, 'phase']

    for otype in range(0, len(outputtypes)):
        tf_dat = mne.time_frequency.tfr_morlet(
            data,
            freqs=freqs,
            n_cycles=n_cycles,
            use_fft=False,
            decim=1,
            n_jobs=None,
            return_itc = False,
            average=False,
            output = outputtypes[otype]
        )
        
        #output_tfr_array(pp, tf_dat, ppdir, output = outputtypes[otype])

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1495.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    6.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:  1.8min finished


Not setting metadata


In [ ]:
#out_fname = os.path.join(ppdir, 'p%03d_%s_tfr_array.npy' % (pp, 'phase'))
#ab = np.load(out_fname)